## Imports and Setup

In [ ]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('/home/itingzon.unicef/giga')
from src import sat_download
from utils import post_utils
from utils import calib_utils
from utils import config_utils
from utils import data_utils
from utils import model_utils
from utils import plot_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [253]:
iso_code = "MOZ"
sum_threshold=-1
buffer_size=25
calibrator=None
cam_method="gradcamelementwise"

data_config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AF.yaml")
model_config_file = os.path.join(os.getcwd(), "configs/best_models.yaml")
data_config = config_utils.load_config(data_config_file)
model_config = config_utils.load_config(model_config_file)

## Load Model Predictions

In [258]:
preds = post_utils.load_preds(
    iso_code, data_config, model_config, cam_method, sum_threshold, buffer_size, calibrator
)
preds.head(2)

Reading files for MOZ...: 100%|████████████████████| 159/159 [00:14<00:00, 10.71it/s]                                                
INFO:root:Data dimensions: (43420, 9)


,prob,UID,sum,geometry,group,ADM1,ADM2,ADM3,PUID
0,0.98088,7658,62.0,POINT (3725889.788 -2169592.396),3041,Manica,Cidade De Chimoio,Cidade De Chimoio,Cidade De Chimoio_7658
1,0.98079,24801,50.0,POINT (3882453.658 -2246136.740),21603,Sofala,Cidade Da Beira,Cidade Da Beira,Cidade Da Beira_24801


## Load Government Data

In [259]:
master = post_utils.load_master(iso_code, data_config)
master.head(2)

INFO:root:clean
0    9958
1    1771
2     722
Name: count, dtype: int64
INFO:root:Data dimensions: (12451, 13)


,MUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,UNICEF-MOZ-SCHOOL-00000000,UNICEF,MOZ,Mozambique,Africa,Sub-Saharan Africa,None,0,0,POINT (35.10153 -17.41236),Tete,Mutarara,Nhamayabue
1,UNICEF-MOZ-SCHOOL-00000001,UNICEF,MOZ,Mozambique,Africa,Sub-Saharan Africa,None,1,0,POINT (36.91100 -14.50489),Niassa,Metarica,Nacumua


## Load OSM and Overture Data

In [260]:
osm_overture = post_utils.load_osm_overture(iso_code, data_config)
osm_overture.head(2)

INFO:root:clean
0    487
Name: count, dtype: int64
INFO:root:Data dimensions: (487, 13)


,SUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,OSM-MOZ-SCHOOL-00000000,OSM,MOZ,Mozambique,Africa,Sub-Saharan Africa,primary school,0,0,POINT (34.75546 -19.64221),Sofala,Dondo,Dondo
1,OSM-MOZ-SCHOOL-00000001,OSM,MOZ,Mozambique,Africa,Sub-Saharan Africa,Maputo International School,1,0,POINT (32.58823 -25.97934),Maputo City,Cidade De Maputo,Cidade De Maputo


## Save Results

In [262]:
post_utils.save_results(iso_code, preds, model_config, cam_method=cam_method, source="preds")
post_utils.save_results(iso_code, master, model_config, source="master")
post_utils.save_results(iso_code, osm_overture, model_config, source="osm_overture")

## Standardize Data

In [270]:
preds = post_utils.standardize_data(model_config, iso_code, cam_method=cam_method, source="preds", uid="UID")
master = post_utils.standardize_data(model_config, iso_code, source="master", uid="MUID")
osm = post_utils.standardize_data(model_config, iso_code, source="osm_overture", uid="SUID")
preds.head(3)

,giga_id_school,school_id,name,lat,lon,admin1,admin1_id_giga,admin2,admin2_id_giga,education_level,source,uninhabited,is_duplicated,predicted_proba,rurban
0,ML-MOZ-SCHOOL-76580000,ML-MOZ-SCHOOL-76580000,Unknown,-19.124442,33.470237,Manica,MOZ006,Cidade de Chimoio,MOZ006001,None,ML,None,None,0.980880,urban
1,ML-MOZ-SCHOOL-24801000,ML-MOZ-SCHOOL-24801000,Unknown,-19.772812,34.876675,Sofala,MOZ007,Cidade da Beira,MOZ007001,None,ML,None,None,0.980790,urban
2,ML-MOZ-SCHOOL-21286000,ML-MOZ-SCHOOL-21286000,Unknown,-19.748836,34.851200,Sofala,MOZ007,Cidade da Beira,MOZ007001,None,ML,None,None,0.979622,urban


In [264]:
data = pd.concat([preds, master, osm])
data = data.reset_index(drop=True)
data.to_parquet(f'{iso_code}_validator_meta.parquet')

/tmp/ipykernel_675940/205928062.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([preds, master, osm])
